In [1]:
import pandas as pd
from pathlib import Path
#from updates_join_annotations import open_df_from_tsv
#import updates_get_average_scores_per_label as scores


In [2]:
# Read in file and set data type for two columns
filepath = './sample_data/label_level_df_sorted.xlsx'
df = pd.read_excel(filepath, dtype={'tokens': list, 'annotators':set})

In [3]:
# Sort values and reset index to allow looping later
df.sort_values(by=['file_id', 'start_index', 'end_index', 'Label 1'], inplace=True)
df.reset_index(inplace=True, drop=True)
df.head()

,file_id,start_index,end_index,Label 1,tokens,annotators,annotator_count
0,1234,6,8,Stemming,"['paniek', 'aanvallen']","{'d', 'e'}",2
1,1234,8,9,Stemming,['token'],"{'c', 'd', 'e'}",3
2,1234,9,10,Stemming,"['token', 'token']",{'b'},1
3,1234,15,15,Stemming,['token'],"{'c', 'd', 'e'}",3
4,1234,15,16,Stemming,"['token', 'token']",{'b'},1


In [5]:
#file_id = 2345
#label = 'STM0'
label_dict = dict()

# Loop through file_ids (change later)
#for file_id in set(df['file_id']):
for file_id in [1234]:
    # Create filtered df
    df_filtered = df[df['file_id'] == file_id]
    # Loop through labels and create further filtered df
    for label in set(df_filtered['Label 1']):
        
        counter = len(label_dict)+1
        
        print(file_id, label)
        
        df_label = df_filtered[df_filtered['Label 1'] == label]
        df_label.reset_index(inplace=True, drop=True)
        
        
        # Set start and end index values
        start_index = df_label['start_index'][0]
        end_index = df_label['end_index'][0]
        
        # If only one tag of label in this file
        if df_label.shape[0] == 1:
            counter = len(label_dict)+1
            label_dict[counter] = {'file_id': file_id,
                                           'start_index': start_index, 
                                           'end_index': end_index,
                                           'label': label,
                                           'tokens': row['tokens'],
                                           'annotators': row['annotators']}
            continue
        
        # label through rest of df
        for index, row in df_label[1:].iterrows():
            # Note: df which it is looping over contains only data from one file from one label

            start_temp = row['start_index']
            end_temp = row['end_index']
            print('temp', start_temp, end_temp)

            current_range = range(start_index, end_index+1)

            # Since the df_label is sorted on start & end index, only a few cases need to be checked
            # If the temporary start vallue is in the range of the saved start value
            if start_temp in current_range:
                print('included')
                if end_temp not in current_range: # And the temp end value is not, then update the end value
                    print('updated')
                    end_index = end_temp
                # If the temp end value is also in the range then do nothing and move on.
            else:
                print('else, stored')
                if counter in label_dict:
                    label_dict[counter]['end_index'] = end_index
                    label_dict[counter]['annotators'].add(row['annotators'])
                    
                else: 
                    label_dict[counter] = {'file_id': file_id,
                                           'start_index': start_index, 
                                           'end_index': end_index,
                                           'label': label,
                                           'tokens': row['tokens'],
                                           'annotators': row['annotators']}

            # For last entry
            if index == len(df_label)-1:
                print('last, stored')
                if counter in label_dict:
                    label_dict[counter]['end_index'] = end_index
                    label_dict[counter]['annotators'].add(row['annotators'])
                    
                else: 
                    label_dict[counter] = {'file_id': file_id,
                                           'start_index': start_index, 
                                           'end_index': end_index,
                                           'label': label,
                                           'tokens': row['tokens'],
                                           'annotators': row['annotators']}
    
    
                    
label_dict   

1234 Stemming
temp 8 9
included
updated
temp 9 10
included
updated
temp 15 15
else, stored
temp 15 16
else, stored


AttributeError: 'str' object has no attribute 'add'

In [ ]:
df_label['annotators'][0]

In [ ]:
# file_id = 1234
# label = 'Stemming'
# label_dict = {label: dict()}
# # Set impossible start and end index values
# start_index = -2
# end_index = -1

# for index, row in df_label.iterrows():
#     # Note: df which it is looping over contains only data from one file from one label
#     #counter = len(label_dict[label])+1
    
#     start_temp = row['start_index']
#     end_temp = row['end_index']
#     print('temp', start_temp, end_temp)
    
#     current_range = range(start_index, end_index+1)
    
#     # If temp start and end are in currently stored start-end range, change nothing
# #     if start_temp in current_range and end_temp in current_range:
# #         continue
#     # If temp start in the current range (there is overlap), and end temp is not in the current range, update the end index
#     if start_temp in current_range and not end_temp in current_range:
#         print('a')
#         end_index = end_temp
#     # Elif temp end in the current range (there is overlap), and start temp is not in the current range, update the start index
#     elif end_temp in current_range and not start_temp in current_range:
#         print('b')
#         start_index = start_temp
#     # Elif if the temp range encompasses the stored/current range, update both start and end indices
#     elif start_index in range(start_temp, end_temp+1) and end_index in range(start_temp, end_temp+1):
#         print('c')
#         start_index = start_temp
#         end_index = end_temp
        
#     elif all([not start_temp in current_range,
#               not end_temp in current_range,
#               start_index >= 0,  # As to not save initial values
#               end_index >=0]): 
#         # If new start and end indices are disjoint from stored indices, then save and restart
#         print('d')
#         counter = len(label_dict[label])+1
#         label_dict[label][counter] = (start_index, end_index)
#         start_index = start_temp
#         end_index = end_temp
        
#     if index == len(df_label)-1:
#         print('yes')
#         counter = len(label_dict[label])+1
#         label_dict[label][counter] = (start_index, end_index)
#     print('stored', start_index, end_index)